# import #


In [1]:
import json
import hashlib
import pandas as pd
from pandas.util import hash_pandas_object
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.api_core.exceptions import GoogleAPICallError
import schema

## import configuration ##

In [2]:
environment = 'prod'
try:
    with open("env") as f:
        environment = f.read()
finally:
    pass
    
environment

'test'

In [3]:
if environment == 'test':
    config_file = "configuration-test.json"
else:
    config_file = "configuration.json"

with open(config_file) as f:
    config = json.load(f)

config_file

'configuration-test.json'

# connect to database #

In [4]:
service_account_file = config["google_account_auth"]
credentials = service_account.Credentials.from_service_account_file(
    service_account_file,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [5]:
dataset_id = config['dataset_id']

## upload data md_customers ##

load customers data

In [6]:
customers_df = pd.read_pickle('./data/customers.pkl').sort_values(by='CUSTOMER_INTERNAL_ID').fillna('')
customers_df.head()

,CUSTOMER_ID,CUSTOMER_NAME,CUSTOMER_INTERNAL_ID,CUSTOMER_FOLDER_NAME,PILOT,IS_ACTIVE,HAS_CONTRACT,HAS_RFQ
18,000039_FCLI,BONDUELLE,003,Bonduelle 003,Damien VAN RYSSEL,yes,yes,
19,000519_FCLI,BOSCH,004,Bosch 004,Sylvie PIEROTTI,yes,yes,
118,,Lactalis,005,Lactalis 005,,,,
119,,Lafarge Holcim,006,Lafarge Holcim 006,,,,
60,000514_FCLI,NESTLE,007,Nestle 007,Philippe HEBERT,,,


In [7]:
customers_df['IS_ACTIVE_bool'] = customers_df['IS_ACTIVE'].map(lambda x: True if x.lower() == 'yes' else False)
customers_df['HAS_CONTRACT_bool'] = customers_df['HAS_CONTRACT'].map(lambda x: True if x.lower() == 'yes' else False)
customers_df['HAS_RFQ_bool'] = customers_df['HAS_RFQ'].map(lambda x: True if x.lower() == 'yes' else False)
customers_df = customers_df[['CUSTOMER_INTERNAL_ID', 'CUSTOMER_NAME', 'CUSTOMER_FOLDER_NAME', 'PILOT', 'IS_ACTIVE_bool', 'HAS_CONTRACT_bool', 'HAS_RFQ_bool']]
customers_df['CUSTOMER_NAME'] = customers_df['CUSTOMER_NAME'].str.upper()
customers_df.head()

,CUSTOMER_INTERNAL_ID,CUSTOMER_NAME,CUSTOMER_FOLDER_NAME,PILOT,IS_ACTIVE_bool,HAS_CONTRACT_bool,HAS_RFQ_bool
18,003,BONDUELLE,Bonduelle 003,Damien VAN RYSSEL,True,True,False
19,004,BOSCH,Bosch 004,Sylvie PIEROTTI,True,True,False
118,005,LACTALIS,Lactalis 005,,False,False,False
119,006,LAFARGE HOLCIM,Lafarge Holcim 006,,False,False,False
60,007,NESTLE,Nestle 007,Philippe HEBERT,False,False,False


In [8]:
cs_customers = hashlib.md5(pd.util.hash_pandas_object(customers_df, index=False).values).hexdigest()
cs_customers

'c89e30fea7b132902f135585c230c6fa'

In [9]:
table_id = dataset_id + "." + config['md_customers_table']

In [10]:
df = client.list_rows(table_id).to_dataframe().sort_values(by='customer_id').fillna('')
df.head()

E0000 00:00:1767605142.120320  403640 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


,customer_id,customer_name,customer_folder_name,pilot,is_active,has_contract,has_rfq
124,003,BONDUELLE,Bonduelle 003,Damien VAN RYSSEL,True,True,False
133,004,BOSCH,Bosch 004,Sylvie PIEROTTI,True,True,False
0,005,LACTALIS,Lactalis 005,,False,False,False
1,006,LAFARGE HOLCIM,Lafarge Holcim 006,,False,False,False
55,007,NESTLE,Nestle 007,Philippe HEBERT,False,False,False


In [11]:
checksum = hashlib.md5(pd.util.hash_pandas_object(df, index=False).values).hexdigest()
checksum

'614c380aa78bc87da142b60956ac7955'

In [12]:
if cs_customers != checksum:
    # delete all entries
    dml_statement = 'DELETE FROM ' + table_id + ' WHERE TRUE'

    query_job = client.query(dml_statement)
    query_job.result()

    # upload new data
    job_config = bigquery.LoadJobConfig(
        schema=schema.schema_customers,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    )

    json_data = []
    for index, row in customers_df.iterrows():  
        json_entry = {}
        json_entry['customer_id'] = row['CUSTOMER_INTERNAL_ID']
        json_entry['customer_name'] = row['CUSTOMER_NAME']
        json_entry['customer_folder_name'] = row['CUSTOMER_FOLDER_NAME']
        json_entry['pilot'] = row['PILOT']
        json_entry['is_active'] = row['IS_ACTIVE_bool']
        json_entry['has_contract'] = row['HAS_CONTRACT_bool']
        json_entry['has_rfq'] = row['HAS_RFQ_bool']
        
        json_data.append(json_entry)

    try:
        load_job = client.load_table_from_json(
            json_data,
            table_id,
            job_config=job_config,
        )
        result = load_job.result()
        print(f"{table_id}: upload successfully")
    except GoogleAPICallError as e:
        print(f"{table_id}: error: upload failed: {e}")

else:
    print(f"{table_id}: is up to date")

one-crm-test-dev.ONE_MRO.md_customers: upload successfully


## upload data md_plants ##

load plants data

In [13]:
plants_df = pd.read_pickle('./data/plants.pkl').sort_values(by='PLANT_ID').fillna('')
plants_df.head()

,CUSTOMER_ID,PLANT_ID,PLANT_NAME,COUNTRY,CITY,SECTOR,CUSTOMER_INTERNAL_ID,PLANT_CLOSED
157,000039_FCLI,000714_FCLI,BONDUELLE - BFE - FR - 69 - GENAS,FR,GENAS,FOOD AND BEVERAGE,003,
150,000039_FCLI,000716_FCLI,BONDUELLE - BFE - FR - 29 - ROSPORDEN,FR,ROSPORDEN,FOOD AND BEVERAGE,003,
146,000039_FCLI,000718_FCLI,BONDUELLE - SOLEAL - FR - 40 - BORDERES,FR,BORDERES,FOOD AND BEVERAGE,003,
138,000039_FCLI,000721_FCLI,BONDUELLE - BELL - BE - 8610 - KORTEMARK,BE,KORTEMARK,FOOD AND BEVERAGE,003,
147,000039_FCLI,000722_FCLI,BONDUELLE - SOLEAL - FR - 40 - LABENNE,FR,LABENNE,FOOD AND BEVERAGE,003,


In [14]:
plants_df['PLANT_CLOSED_bool'] = plants_df['PLANT_CLOSED'].map(lambda x: True if x.lower() == 'yes' else False)
plants_df = plants_df[['CUSTOMER_INTERNAL_ID', 'PLANT_ID', 'PLANT_NAME', 'COUNTRY', 'CITY', 'SECTOR', 'PLANT_CLOSED_bool']]
plants_df.head()

,CUSTOMER_INTERNAL_ID,PLANT_ID,PLANT_NAME,COUNTRY,CITY,SECTOR,PLANT_CLOSED_bool
157,003,000714_FCLI,BONDUELLE - BFE - FR - 69 - GENAS,FR,GENAS,FOOD AND BEVERAGE,False
150,003,000716_FCLI,BONDUELLE - BFE - FR - 29 - ROSPORDEN,FR,ROSPORDEN,FOOD AND BEVERAGE,False
146,003,000718_FCLI,BONDUELLE - SOLEAL - FR - 40 - BORDERES,FR,BORDERES,FOOD AND BEVERAGE,False
138,003,000721_FCLI,BONDUELLE - BELL - BE - 8610 - KORTEMARK,BE,KORTEMARK,FOOD AND BEVERAGE,False
147,003,000722_FCLI,BONDUELLE - SOLEAL - FR - 40 - LABENNE,FR,LABENNE,FOOD AND BEVERAGE,False


In [15]:
cs_plants = hashlib.md5(pd.util.hash_pandas_object(plants_df, index=False).values).hexdigest()
cs_plants

'e60a35b8621e5097c4d840b6d4b64f0d'

upload plants data to database

In [16]:
table_id = dataset_id + "." + config['md_plants_table']

In [17]:
df = client.list_rows(table_id).to_dataframe().sort_values(by='plant_id').fillna('')
df.head()

E0000 00:00:1767605241.334164  403640 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


,customer_id,plant_id,plant_name,country,city,sector,plant_closed


In [18]:
checksum = hashlib.md5(pd.util.hash_pandas_object(df, index=False).values).hexdigest()
checksum

'd41d8cd98f00b204e9800998ecf8427e'

In [19]:
if cs_plants != checksum:
    # delete all entries
    dml_statement = 'DELETE FROM ' + table_id + ' WHERE TRUE'

    query_job = client.query(dml_statement)
    query_job.result()

    # upload new data
    job_config = bigquery.LoadJobConfig(
        schema=schema.schema_plants,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    )

    json_data = []
    for index, row in plants_df.iterrows():  
        json_entry = {}
        json_entry['customer_id'] = row['CUSTOMER_INTERNAL_ID']
        json_entry['plant_id'] = row['PLANT_ID']
        json_entry['plant_name'] = row['PLANT_NAME']
        json_entry['country'] = row['COUNTRY']
        json_entry['city'] = row['CITY']
        json_entry['sector'] = row['SECTOR']
        json_entry['plant_closed'] = row['PLANT_CLOSED_bool']        

        json_data.append(json_entry)

    try:
        load_job = client.load_table_from_json(
            json_data,
            table_id,
            job_config=job_config,
        )
        result = load_job.result()
        print(f"{table_id}: upload successfully")
    except GoogleAPICallError as e:
        print(f"{table_id}: error: upload failed: {e}")

else:
    print(f"{table_id}: is up to date")

one-crm-test-dev.ONE_MRO.md_plants: upload successfully


## upload md_productfamilies ##

load productfamlilies data

In [20]:
productfamilies_df = pd.read_pickle('./data/productfamilies.pkl').sort_values(by='PRODUCTFAMILY_ID').fillna('')
productfamilies_df.head()

,PRODUCTFAMILY_ID,PRODUCTFAMILY_CODE,PRODUCTFAMILY_NAME
0,01_Roulements,01,"Bearings (bearing, housing)"
1,02_Transmission mecanique,02,Mechanical Transmission
2,03_Transmission electromecanique,03,Electromechanically transmission (motors …)
3,04_Guidage,04,Linear motion
4,05_Etancheite,05,Sealing


In [21]:
cs_productfamilies = hashlib.md5(pd.util.hash_pandas_object(productfamilies_df, index=False).values).hexdigest()
cs_productfamilies

'ee6a104d00a4c26fd1d2355f79d9bae8'

upload productfamilies data to database

In [22]:
table_id = dataset_id + "." + config['md_productfamilies_table']

In [23]:
df = client.list_rows(table_id).to_dataframe().sort_values(by='productfamily_id').fillna('')
df.head()

E0000 00:00:1767605284.988221  403640 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


,productfamily_id,productfamily_code,productfamily_name
5,01_Roulements,01,"Bearings (bearing, housing)"
11,02_Transmission mecanique,02,Mechanical Transmission
7,03_Transmission electromecanique,03,Electromechanically transmission (motors …)
10,04_Guidage,04,Linear motion
15,05_Etancheite,05,Sealing


In [24]:
checksum = hashlib.md5(pd.util.hash_pandas_object(df, index=False).values).hexdigest()
checksum

'f5efdb267b3618afc7be25a5ec249aea'

In [25]:
if cs_productfamilies != checksum:
    # delete all entries
    dml_statement = 'DELETE FROM ' + table_id + ' WHERE TRUE'

    query_job = client.query(dml_statement)
    query_job.result()

    # upload new data
    job_config = bigquery.LoadJobConfig(
        schema=schema.schema_productfamilies,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    )

    json_data = []
    for index, row in productfamilies_df.iterrows():  
        json_entry = {}
        json_entry['productfamily_id'] = row['PRODUCTFAMILY_ID']
        json_entry['productfamily_code'] = row['PRODUCTFAMILY_CODE']
        json_entry['productfamily_name'] = row['PRODUCTFAMILY_NAME']

        json_data.append(json_entry)

    try:
        load_job = client.load_table_from_json(
            json_data,
            table_id,
            job_config=job_config,
        )
        result = load_job.result()
        print(f"{table_id}: upload successfully")
    except GoogleAPICallError as e:
        print(f"{table_id}: error: upload failed: {e}")

else:
    print(f"{table_id}: is up to date")

one-crm-test-dev.ONE_MRO.md_productfamilies: upload successfully


## upload md_members ##

load members data

In [26]:
members_df = pd.read_pickle('./data/members.pkl').sort_values(by='MEMBER_ID').fillna('')
members_df.head()

,MEMBER_ID,MEMBER_NAME,COUNTRY
21,000090_FMEM,ONE GmbH,DE
12,000091_FMEM,Groupe FAIR,FR
24,000092_FMEM,PARTOOL,BE
13,000093_FMEM,HAAS UND KELLHOFER,DE
19,000094_FMEM,LUDWIG MEISTER,DE


In [27]:
cs_members = hashlib.md5(pd.util.hash_pandas_object(members_df, index=False).values).hexdigest()
cs_members

'6159b7c7322c28a15aec8ed01ba1a9ae'

upload members data to database

In [28]:
table_id = dataset_id + "." + config['md_members_table']

In [29]:
df = client.list_rows(table_id).to_dataframe().sort_values(by='member_id').fillna('')
df.head()

E0000 00:00:1767605305.262635  403640 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


,member_id,member_name,country
1,000090_FMEM,ONE GmbH,DE
9,000091_FMEM,Groupe FAIR,FR
0,000092_FMEM,PARTOOL,BE
2,000093_FMEM,HAAS UND KELLHOFER,DE
3,000094_FMEM,LUDWIG MEISTER,DE


In [30]:
checksum = hashlib.md5(pd.util.hash_pandas_object(df, index=False).values).hexdigest()
checksum

'6159b7c7322c28a15aec8ed01ba1a9ae'

In [31]:
if cs_members != checksum:
    # delete all entries
    dml_statement = 'DELETE FROM ' + table_id + ' WHERE TRUE'

    query_job = client.query(dml_statement)
    query_job.result()

    # upload new data
    job_config = bigquery.LoadJobConfig(
        schema=schema.schema_members,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    )

    json_data = []
    for index, row in members_df.iterrows():  
        json_entry = {}
        json_entry['member_id'] = row['MEMBER_ID']
        json_entry['member_name'] = row['MEMBER_NAME']
        json_entry['country'] = row['COUNTRY']

        json_data.append(json_entry)

    try:
        load_job = client.load_table_from_json(
            json_data,
            table_id,
            job_config=job_config,
        )
        result = load_job.result()
        print(f"{table_id}: upload successfully")
    except GoogleAPICallError as e:
        print(f"{table_id}: error: upload failed: {e}")

else:
    print(f"{table_id}: is up to date")

one-crm-test-dev.ONE_MRO.md_members: is up to date


## upload md_branches ##

load branches data

In [32]:
branches_df = pd.read_pickle('./data/branches.pkl').sort_values(by='BRANCH_ID').fillna('')
branches_df.head()

,MEMBER_ID,BRANCH_ID,BRANCH_NAME,MEMBER_NAME,BRANCH_CLOSED
153,000115_FMEM,ABCONOT,ABCO - Nottingham,ABCO,
86,000109_FMEM,BIEALE,BIERREDI - Alessandria,BIERREDI,
87,000109_FMEM,BIECUNEO,BIERREDI - Cuneo,BIERREDI,
85,000109_FMEM,BIEGEN,BIERREDI - Genova,BIERREDI,
89,000109_FMEM,BIESAV,BIERREDI - Savona,BIERREDI,


In [33]:
branches_df['BRANCH_CLOSED_bool'] = branches_df['BRANCH_CLOSED'].map(lambda x: True if x.lower() == 'yes' else False)
branches_df = branches_df[['MEMBER_ID', 'BRANCH_ID', 'BRANCH_NAME', 'MEMBER_NAME', 'BRANCH_CLOSED_bool']]
branches_df.head()

,MEMBER_ID,BRANCH_ID,BRANCH_NAME,MEMBER_NAME,BRANCH_CLOSED_bool
153,000115_FMEM,ABCONOT,ABCO - Nottingham,ABCO,False
86,000109_FMEM,BIEALE,BIERREDI - Alessandria,BIERREDI,False
87,000109_FMEM,BIECUNEO,BIERREDI - Cuneo,BIERREDI,False
85,000109_FMEM,BIEGEN,BIERREDI - Genova,BIERREDI,False
89,000109_FMEM,BIESAV,BIERREDI - Savona,BIERREDI,False


In [34]:
cs_branches = hashlib.md5(pd.util.hash_pandas_object(branches_df, index=False).values).hexdigest()
cs_branches

'c20afb3af2b000ae0a0905953d77a2e6'

upload branches to database

In [35]:
table_id = dataset_id + "." + config['md_branches_table']

In [36]:
df = client.list_rows(table_id).to_dataframe().sort_values(by='branch_id').fillna('')
df.head()

E0000 00:00:1767605406.789194  403640 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


,member_id,branch_id,branch_name,member_name,branch_closed


In [37]:
checksum = hashlib.md5(pd.util.hash_pandas_object(df, index=False).values).hexdigest()
checksum

'd41d8cd98f00b204e9800998ecf8427e'

In [38]:
if cs_branches != checksum:
    # delete all entries
    dml_statement = 'DELETE FROM ' + table_id + ' WHERE TRUE'

    query_job = client.query(dml_statement)
    query_job.result()

    # upload new data
    job_config = bigquery.LoadJobConfig(
        schema=schema.schema_branches,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    )

    json_data = []
    for index, row in branches_df.iterrows():  
        json_entry = {}
        json_entry['branch_id'] = row['BRANCH_ID']
        json_entry['branch_name'] = row['BRANCH_NAME']
        json_entry['member_id'] = row['MEMBER_ID']
        json_entry['member_name'] = row['MEMBER_NAME']
        json_entry['branch_closed'] = row['BRANCH_CLOSED_bool']

        json_data.append(json_entry)

    try:
        load_job = client.load_table_from_json(
            json_data,
            table_id,
            job_config=job_config,
        )
        result = load_job.result()
        print(f"{table_id}: upload successfully")
    except GoogleAPICallError as e:
        print(f"{table_id}: error: upload failed: {e}")

else:
    print(f"{table_id}: is up to date")

one-crm-test-dev.ONE_MRO.md_branches: upload successfully


## upload md_suppliers ##

load suppliers data

In [39]:
suppliers_df = pd.read_pickle('./data/suppliers.pkl').sort_values(by='SUPPLIER_ID').fillna('')
suppliers_df.head()

,SUPPLIER_ID,SUPPLIER_NAME
80,000001_FFOUR,BOURDON
502,000002_FFOUR,RENOLD (BRAMPTON)
86,000003_FFOUR,BURGMANN
87,000004_FFOUR,BURKERT
110,000006_FFOUR,CITEC


In [40]:
cs_suppliers = hashlib.md5(pd.util.hash_pandas_object(suppliers_df, index=False).values).hexdigest()
cs_suppliers

'ac9451155230bb2429335d4f43336b45'

upload suppliers data to database

In [41]:
table_id = dataset_id + "." + config['md_suppliers_table']

In [42]:
df = client.list_rows(table_id).to_dataframe().sort_values(by='supplier_id').fillna('')
df.head()

E0000 00:00:1767605443.508289  403640 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


,supplier_id,supplier_name
79,000001_FFOUR,BOURDON
502,000002_FFOUR,RENOLD (BRAMPTON)
85,000003_FFOUR,BURGMANN
86,000004_FFOUR,BURKERT
109,000006_FFOUR,CITEC


In [43]:
checksum = hashlib.md5(pd.util.hash_pandas_object(df, index=False).values).hexdigest()
checksum

'ac9451155230bb2429335d4f43336b45'

In [44]:
if cs_suppliers != checksum:
    # delete all entries
    dml_statement = 'DELETE FROM ' + table_id + ' WHERE TRUE'

    query_job = client.query(dml_statement)
    query_job.result()

    # upload new data
    job_config = bigquery.LoadJobConfig(
        schema=schema.schema_suppliers,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    )

    json_data = []
    for index, row in suppliers_df.iterrows():  
        json_entry = {}
        json_entry['supplier_id'] = row['SUPPLIER_ID']
        json_entry['supplier_name'] = row['SUPPLIER_NAME']

        json_data.append(json_entry)

    try:
        load_job = client.load_table_from_json(
            json_data,
            table_id,
            job_config=job_config,
        )
        result = load_job.result()
        print(f"{table_id}: upload successfully")
    except GoogleAPICallError as e:
        print(f"{table_id}: error: upload failed: {e}")

else:
    print(f"{table_id}: is up to date")

one-crm-test-dev.ONE_MRO.md_suppliers: is up to date


In [45]:
client.close()